In [ ]:
import pandas as pd
import requests
import zipfile
import io
import os
import csv
import ctypes as ct
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import re


- "https://www.bankofengland.co.uk/-/media/boe/files/markets/asset-purchase-facility/gilt-purchase-operational-results.xlsx"
- "https://www.bankofengland.co.uk/-/media/boe/files/markets/asset-purchase-facility/gilt-sales-time-series.xlsx"

In [ ]:
df_buy = pd.ExcelFile("downloads/gilt-purchase-operational-results.XLSX")

In [ ]:
df_buy.sheet_names

In [ ]:
df_buy = pd.read_excel("downloads/gilt-purchase-operational-results.XLSX", sheet_name="APF Gilts", header=1)

In [ ]:
df_buy.columns

In [ ]:
df_boe = df_buy[['Operation date','Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)']].copy()

In [ ]:
df_boe['Operation date'] = pd.to_datetime(df_boe['Operation date'], format='%d-%m-%Y')

In [ ]:
df_boe.set_index('Operation date', inplace=True)

In [ ]:
df_boe.cumsum(axis=0).plot()

In [ ]:
df_sell = pd.ExcelFile("downloads/gilt-sales-time-series.XLSX")

In [ ]:
df_sell.sheet_names

In [ ]:
df_sell = pd.read_excel("downloads/gilt-sales-time-series.XLSX", sheet_name="APF gilt sales", header=1)

In [ ]:
df_sell.columns

In [ ]:
df_boe_exit = df_sell[['Operation date','Total allocation (proceeds £mn)', 'Total allocation (nominal £mn)']].copy()
df_boe_exit['Operation date'] = pd.to_datetime(df_boe_exit['Operation date'], format='%d-%m-%Y')
df_boe_exit.set_index('Operation date', inplace=True)
df_boe_exit.cumsum(axis=0).plot()

In [ ]:
bonds = df_buy[['ISIN','Bond\n']].drop_duplicates()

In [ ]:
from collections import defaultdict
isin_bond_map = defaultdict(list)
for row in bonds.itertuples():
    isin_bond_map[row.ISIN].append(row._2)
for isin, bond in isin_bond_map.items():
    if len(bond) > 1:
        raise Exception(f"There are duplicates in the BoE list: {isin}")
isin_bond_map['GB0008881541']

In [ ]:
gilt_level_buys = df_buy[[
    'ISIN',
    'Operation date',
    'Total allocation (proceeds £mn)',
    'Total allocation (nominal £mn)'
    ]].copy()
gilt_level_buys.set_index('Operation date', inplace=True)
gilt_level_buys.index = pd.to_datetime(gilt_level_buys.index, format='%d-%m-%Y').to_period('M').to_timestamp('M')
gilt_level_buys.reset_index(inplace=True)


In [ ]:
gilt_buys_ts = gilt_level_buys.pivot_table(
    index='Operation date', 
    columns='ISIN', 
    values='Total allocation (nominal £mn)', aggfunc='sum')

In [ ]:
gilt_buys_ts.columns

In [ ]:
gilt_buys_ts = gilt_buys_ts.fillna(0).cumsum()
gilt_buys_ts.head()
# TODO: Clip once the Bond has matured

In [ ]:
gilt_buys_total = gilt_level_buys.reset_index().groupby([ 'ISIN']).agg(
    Total_Allocation_Proceeds=('Total allocation (proceeds £mn)', 'sum'),
    Total_Allocation_Nominal=('Total allocation (nominal £mn)', 'sum'),
    Last_Purchase_Month=('Operation date', 'max')
)
gilt_buys_total.reset_index(inplace=True)
gilt_buys_total.head()
    

In [ ]:
from bgs.load_gilt_details import load_csv_blocks
details = load_csv_blocks("downloads/BGSDetails.csv")

In [ ]:
conv_details = details['Conventionals']

In [ ]:
conv_details.columns

In [ ]:
conv_details.head()

In [ ]:
isin_bond_details = conv_details[['ISIN Code', 'Latest redemption date','%']].copy()

In [ ]:
[x for x in gilt_buys_total['ISIN'] if x not in isin_bond_details['ISIN Code'].to_list()]

In [ ]:
gilt_buys_total['ISIN'][0]

In [ ]:
isin_bond_details[isin_bond_details['ISIN Code']=='GB0002404191']['Latest redemption date'].values[0]

In [ ]:
gilt_buys_total['maturity_date'] = gilt_buys_total['ISIN'].map(
    lambda x: isin_bond_details[
        isin_bond_details['ISIN Code']==x
        ]['Latest redemption date'].values[0]
    )

In [ ]:
gilt_buys_total['maturity_date'] = pd.to_datetime(
    gilt_buys_total['maturity_date'], format="%d %b %Y"
)

In [ ]:
matured = gilt_buys_total['maturity_date'] > pd.Timestamp.now()

In [ ]:
boe_portfolio = gilt_buys_total[matured].copy()

In [ ]:
boe_portfolio['Coupon']=boe_portfolio['ISIN'].map(
    lambda x: isin_bond_details[
        isin_bond_details['ISIN Code']==x
        ]['%'].values[0]
    )

In [ ]:
boe_portfolio.groupby('Coupon')['Total_Allocation_Nominal'].sum().plot(
    kind='bar', figsize=(12, 6), title='BoE Portfolio by Coupon'
)

In [ ]:
boe_portfolio.groupby('Coupon')['Total_Allocation_Nominal'].sum().cumsum()